In [ ]:
!mkdir -p ~/agave/funwave-tvd-jenkins-pipeline

%cd ~/agave/funwave-tvd-jenkins-pipeline

!pip3 install --upgrade setvar

import re
import os
import sys
from setvar import *

# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

!auth-tokens-refresh

# Validating Your Build
* Automation is great, but things can quickly go wrong.
* This is why CI/CD strongly emphasizes good testing practices.
* Testing is central to CI/CD
  * It allows you assess viability of each commit
  * It is the determination of whether or not code can be successfully integrated
  * It allows for code to be automatically deployed to prod, without the direct oversight of a tightly controlled group of developers.
* Production pipelines have multiple testing guards
  * Types of tests: Unit, Functional, Acceptance, Benchmarks, ...

## Adding a Benchmark
* Let's add a simple benchmark to validate performance after each build.
* We'll make a new feature branch for this benchmark

In [ ]:
!ssh sandbox "cd ~/FUNWAVE-TVD && git checkout -b benchmark"

* We're going to need new input files in order to run a strong scaling study

In [ ]:
# [2,1] means PX=1, PY=1,2 total processors
for np in [[1,1], [2,1], [2,2]]:
    writefile("input_{NP}.txt".format(NP=int(np[0]*np[1])),"""
!INPUT FILE FOR FUNWAVE_TVD
  ! NOTE: all input parameter are capital sensitive
  ! --------------------TITLE-------------------------------------
  ! title only for log file
TITLE = VESSEL
  ! -------------------HOT START---------------------------------
HOT_START = F
FileNumber_HOTSTART = 1
  ! -------------------PARALLEL INFO-----------------------------
  !
  !    PX,PY - processor numbers in X and Y
  !    NOTE: make sure consistency with mpirun -np n (px*py)
  !
PX = {PX}
PY = {PY}
  ! --------------------DEPTH-------------------------------------
  ! Depth types, DEPTH_TYPE=DATA: from depth file
  !              DEPTH_TYPE=FLAT: idealized flat, need depth_flat
  !              DEPTH_TYPE=SLOPE: idealized slope,
  !                                 need slope,SLP starting point, Xslp
  !                                 and depth_flat
DEPTH_TYPE = FLAT
DEPTH_FLAT = 10.0
  ! -------------------PRINT---------------------------------
  ! PRINT*,
  ! result folder
RESULT_FOLDER = output/

  ! ------------------DIMENSION-----------------------------
  ! global grid dimension
Mglob = 500
Nglob = 100

  ! ----------------- TIME----------------------------------
  ! time: total computational time/ plot time / screen interval
  ! all in seconds
TOTAL_TIME = 3.0
PLOT_INTV = 1.0
PLOT_INTV_STATION = 50000.0
SCREEN_INTV = 1.0
HOTSTART_INTV = 360000000000.0

WAVEMAKER = INI_GAU
AMP = 3.0
Xc = 250.0
Yc = 50.0
WID = 20.0

  ! -----------------GRID----------------------------------
  ! if use spherical grid, in decimal degrees
  ! cartesian grid sizes
DX = 1.0
DY = 1.0
  ! ----------------SHIP WAKES ----------------------------
VESSEL_FOLDER = ./
NumVessel = 2
  ! -----------------OUTPUT-----------------------------
ETA = T
U = T
V = T
""".format(PX=np[0], PY=np[1]))



Next we'll make directories to store our input files and outputs from benchmarking runs

In [ ]:
!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N /home/jovyan/FUNWAVE-TVD/benchmarks/np_1
!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N /home/jovyan/FUNWAVE-TVD/benchmarks/np_2
!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N /home/jovyan/FUNWAVE-TVD/benchmarks/np_4

Uploading our new input files

In [ ]:
!files-upload -F input_1.txt -S ${AGAVE_STORAGE_SYSTEM_ID} /home/jovyan/FUNWAVE-TVD/benchmarks/np_1/
!files-upload -F input_2.txt -S ${AGAVE_STORAGE_SYSTEM_ID} /home/jovyan/FUNWAVE-TVD/benchmarks/np_2/
!files-upload -F input_4.txt -S ${AGAVE_STORAGE_SYSTEM_ID} /home/jovyan/FUNWAVE-TVD/benchmarks/np_4/

In [ ]:
writefile("funwave-benchmark-wrapper.txt","""

VERSION=$(cat version.txt | paste -sd "." -)

export BASE_DIR=$PWD 
for np in {1,2,4}; do
  cd ${BASE_DIR}/np_${np}
  docker run funwave-tvd:\${VERSION} mpirun -np $np /home/install/FUNWAVE-TVD/src/funwave_vessel
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F funwave-benchmark-wrapper.txt /home/jovyan/FUNWAVE-TVD/build/

In [ ]:
writefile("funwave-benchmark-app.txt","""
{  
   "name":"${AGAVE_USERNAME}-${MACHINE_NAME}-funwave-dbuild",
   "version":"1.0",
   "label":"Benchmarks the funwave docker image",
   "shortDescription":"Funwave docker benchmark",
   "longDescription":"",
   "deploymentSystem":"${AGAVE_STORAGE_SYSTEM_ID}",
   "deploymentPath":"automation/funwave-tvd-docker-automation",
   "templatePath":"build/funwave-benchmark-wrapper.txt",
   "testPath":"test.txt",
   "executionSystem":"${AGAVE_EXECUTION_SYSTEM_ID}",
   "executionType":"CLI",
   "parallelism":"SERIAL",
   "modules":[],
   "inputs":[],
   "parameters":[{
     "id" : "code_version",
     "value" : {
       "visible":true,
       "required":true,
       "type":"string",
       "order":0,
       "enquote":false,
       "default":"latest"
     },
     "details":{
         "label": "Version of the code",
         "description": "If true, output will be packed and compressed",
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     },
     "semantics":{
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     }
   }],
   "outputs":[]
}
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F funwave-benchmark-app.txt /home/jovyan/FUNWAVE-TVD/build/

# Automating Our Benchmark

## Add A Jenkinsfile to Run the Benchmark

In [ ]:
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F /home/jovyan/notebooks/build/Jenkinsfile-benchmark /home/jovyan/FUNWAVE-TVD/build/

# DRYing Out Our Jenkinsfiles
Both our build and benchmark Jenkinsfiles are following the same basic workflow _(checkout code, setup Agave CLI, build and submit job to execution environment)_ which has resulted in a bunch of duplicated code that varies only by a keyword. For the sake of maintainability, we should find a way to improve the orthogonality of our pipeline.

## Jenkins Shared Libraries
A [Jenkins Shared Library](https://jenkins.io/doc/book/pipeline/shared-libraries/) is a repository of code that is centrally managed, and made accessible to multiple pipelines. These pipelines can call shared libraries as functions, and pass parameters to specify build options.

## Update the Build Jenkinsfile to Trigger the Benchmark
* We need to add a new stage to the Jenkins pipeline to run the benchmark if the preceeding stages of the build executed successfully.

In [ ]:
writefile("Jenkinsfile", """import groovy.json.JsonOutput

def JobStages = load("JobStages.groovy")


node {
  currentBuild.result = "SUCCESS"

  env.AGAVE_TENANTS_API_BASEURL = "https://sandbox.agaveplatform.org/tenants"
  env.MACHINE_NAME = "sandbox"
  env.AGAVE_APP_NAME = "funwave-tvd-build-\${env.AGAVE_USERNAME}"
  env.AGAVE_CLIENT_NAME = "jenkins-cli-\${env.AGAVE_USERNAME}"

  try {

    JobStages.setUpCli(env)
    JobStages.cloneRepository()
    JobStages.updateApp('build', env)
    JobStages.submitJob('build', env)

    stage("Run benchmarks") {
      // Benchmarking stuff goes here
    }

  } catch (error) {

    currentBuild.result = "FAILURE"
    throw error

  }
}
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F Jenkinsfile /home/jovyan/FUNWAVE-TVD/build/

# Commit Your Benchmark, Watch It Run
* Let's merge our benchmark back into the `dev` branch and watch it run!

In [ ]:
!ssh sandbox "set -x && cd ~/FUNWAVE-TVD && git add build"
!ssh sandbox "cd ~/FUNWAVE-TVD && git commit -m 'Added benchmark app and pipeline.'"

In [ ]:
!ssh sandbox "cd ~/FUNWAVE-TVD && git checkout dev && git merge --squash benchmark"

# Visualize Our Benchmarks

Let's create functions to gather our data.

In [ ]:
def get_time_from_output(output):
    '''output: list output from funwave split by lines
    ex: ["line1", "line2", ...]
    returns a float of simulation time if it exists'''
    for line in output:
        if "simulation" in line.lower():
            line = ' '.join(line.lower().split())
            split_line = line.split()
            time = float(split_line[2])
            return time
    return "No timing result found!"

In [ ]:
def get_metadata(output):
    '''output: list output from funwave split by lines
    ex: ["line1", "line2", ...]
    returns the date and commit of the run'''
    date = ''
    commit = ''
    for line in output:
        if "funwave run date" in line.lower():
            line = ' '.join(line.lower().split())
            split_line = line.split()
            date = split_line[-1]
        if "funwave commit hash" in line.lower():
            line = ' '.join(line.lower().split())
            split_line = line.split()
            commit = split_line[-1]
    return date + '_' + commit

In [ ]:
def plot_funwave(metadata, results):
    '''metadata: list of date_commits to plot (x-axis)
    ex: ['day1_commit1', 'day2_commit2', 'day3_commit2']
    results: dict of floats containing simulation times from funwave
    ex: results[1] = [57.45267612299358, 58.964640157995746, 57.09651633500471]
    results[2] = [16.213947223004652, 16.57105119198968, 15.723207671995624]
    The lists in results should be the same length as the metadata list!
    '''
    fig, ax = plt.subplots()
    for np in [1, 2, 4]:
        plt.plot( metadata, results[np], marker='o', markersize=12, linewidth=2, label=np)

    plt.title('Funwave Strong Scaling Time vs Commit')
    plt.legend(loc='upper left', title="NP", bbox_to_anchor=(1,1))
    plt.show()